In [1]:
import os

from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from google.cloud import bigquery
from timer import timer

Create client for BigQuery API

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="credentials.json"

client = bigquery.Client()

Retrieve data from public data samples

In [3]:
@timer
def retrieve_big_query():
    with ThreadPoolExecutor() as executor:
        results = []
        for i in range(4):
            query = (f"SELECT * "
                     f"FROM `bigquery-public-data.usa_names.usa_1910_2013`"
                     f"LIMIT 500000 "
                     f"OFFSET {i * 500000}"
                     )
            results.append(executor.submit(client.query, query))
        return [result.result().to_dataframe() for result in results]

dataframes = retrieve_big_query()


Elapsed in 13.937 seconds


Concatenate smaller pieces into one DataFrame

In [4]:
df = pd.concat(dataframes)

Check the data in DataFrame

In [5]:
df.head()

,state,gender,year,name,number
0,AK,F,1910,Mary,14
1,AL,F,1910,Susie,85
2,AL,F,1910,Harriet,8
3,AL,F,1910,Ellie,9
4,AL,F,1910,Amy,9


Check the shape of DataFrame

In [6]:
df.shape

(2000000, 5)

Check if there are any empty cells in DataFrame

In [7]:
df.isnull().sum()

state     0
gender    0
year      0
name      0
number    0
dtype: int64

Separate one DataFrame into list of DataFrames for each state

In [8]:
grouped_dataframes_by_state = df.groupby("state")

Check if data separated correctly

In [9]:
len(grouped_dataframes_by_state)

51

Retrieve the first DatafFrame from grouped collection

In [10]:
first_df = grouped_dataframes_by_state.get_group(list(grouped_dataframes_by_state.groups.keys())[0])

Check the first DataFrame

In [11]:
first_df

,state,gender,year,name,number
0,AK,F,1910,Mary,14
2702,AK,F,1911,Ruth,7
2703,AK,M,1911,George,5
2704,AK,M,1911,James,8
2705,AK,F,1911,Margaret,7
...,...,...,...,...,...
495151,AK,M,1950,Harold,6
495152,AK,M,1950,Fred,7
495153,AK,M,1950,Lawrence,10
495154,AK,M,1950,Dennis,18


In [12]:
first_df.dtypes

state     object
gender    object
year       Int64
name      object
number     Int64
dtype: object

**Write concurrently dataframe into Google Sheet.** (On output we can see the concurrent type of work)
Each DaraFrame in collection occupy its own sheet. Method `write_dataframes_to_sheets` slices first `30_000` rows in order to
fit `10_000_000` limitation for number of cells in Google Sheet.

In [13]:
from writer import write_dataframes_to_sheets

write_dataframes_to_sheets("task_2", grouped_dataframes_by_state)

AK AL AR AZ Done(AK) CA CO CT Done(AL) DC Done(AR) Done(AZ) Done(CA) Done(CO) DE Done(CT) Done(DC) FL GA Done(DE) HI Done(FL) IA Done(GA) ID Done(HI) IL Done(IA) IN Done(ID) KS Done(IL) KY Done(IN) LA Done(KS) MA Done(KY) MD Done(LA) ME Done(MA) MI Done(MD) MN Done(ME) MO Done(MI) MS Done(MN) MT Done(MO) NC Done(MS) ND Done(MT) Done(NC) NE NH Done(ND) NJ Done(NE) NM Done(NH) NV Done(NJ) NY Done(NM) Done(NV) OH OK Done(NY) OR Done(OH) PA Done(OK) RI Done(OR) SC Done(PA) SD Done(RI) TN Done(SC) TX Done(SD) UT Done(TN) VA Done(TX) VT Done(UT) WA Done(VA) Done(VT) WI WV Done(WA) WY Done(WI) Done(WV) Done(WY) 
Elapsed in 657.173 seconds
